## Ideas:

### Attributes

- illumination (one or two) - how does Flamingo converter check these attributes?
- channels
- time points
- positions
- frame interval - diff between last and first time stamp.
- magnification - probably just parse metadata file.

### Methods

- Save max projection
    - Memory converving option: open/save individual frames in dir
    - Full stack option: concatenate MPs into stack and save stack.
- Downsample 3D
    - Memory converving option: open/save individual 3D frames in dir
    - Full stack option: concatenate MPs into stack and save stack.
    - View in Napari True/False
- Combined Max projection and Downsample 3D
    - Computes both at once as a nice saver. Will use slightly more memory but save a lot of time. 